# CCD calculations

In [1]:
import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'

# Force the local gqcpy to be imported
import sys
sys.path.insert(0, '../../build/gqcpy/')

import gqcpy
import numpy as np

np.set_printoptions(precision=3, linewidth=120)

## CCD calculations on H2O // STO-3G

In [2]:
molecule = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/h2o_crawdad.xyz")
N = molecule.numberOfElectrons()
N_P = N // 2

print(molecule)

Number of electrons: 10
O  (0, -0.143226, 0)
H  (1.63804, 1.13655, -0)
H  (-1.63804, 1.13655, -0)



The first task is to generate the RHF reference, so we'll do an RHF SCF calculation.

In [3]:
r_spinor_basis = gqcpy.RSpinorBasis(molecule, "STO-3G")
K = r_spinor_basis.numberOfSpatialOrbitals()

S = r_spinor_basis.quantizeOverlapOperator().parameters()

In [4]:
sq_hamiltonian = gqcpy.SQHamiltonian.Molecular(r_spinor_basis, molecule)  # 'sq' for 'second-quantized'

The current Hamiltionian is expressed in the non-orthogonal AO basis. This is exactly what we need to start an RHF calculation.

In [5]:
environment = gqcpy.RHFSCFEnvironment.WithCoreGuess(N, sq_hamiltonian, S)
solver = gqcpy.RHFSCFSolver.DIIS()

objective = gqcpy.DiagonalRHFFockMatrixObjective(sq_hamiltonian)  # use the default threshold of 1.0e-08

Using this objective makes sure that the optimized expansion coefficients yield a diagonal Fock matrix, so they will represent the canonical RHF spin-orbital basis.

In [6]:
rhf_qc_structure = gqcpy.RHF.optimize(objective, solver, environment)

In [7]:
rhf_energy = rhf_qc_structure.groundStateEnergy() + gqcpy.Operator.NuclearRepulsion(molecule).value()
print(rhf_energy)

-74.94207992809503


This is very much in line with `crawdad`'s results: `-74.942079928192`.

In [8]:
rhf_parameters = rhf_qc_structure.groundStateParameters()
C = rhf_parameters.coefficientMatrix()

In [9]:
r_spinor_basis.transform(C)  # now represents a restricted spin-orbital basis using the canonical RHF MOs

Before continuing with the CCSD specifics, we must first prepare the molecular Hamiltonian in the correct spinor basis. Since we have implemented CCSD using general spinors, we should use a `GSpinorBasis`.

In [10]:
g_spinor_basis = gqcpy.GSpinorBasis.FromRestricted(r_spinor_basis)  # represents a general spinor basis, based off the restricted canonical RHF MOs

M = g_spinor_basis.numberOfSpinors()
ghf_onv = gqcpy.SpinUnresolvedONV.GHF(M, N, rhf_parameters.spinOrbitalEnergiesBlocked())

orbital_space = ghf_onv.orbitalSpace()
print(orbital_space.description())

An orbital space with 14 orbitals.

	The occupied orbital indices are: 0 1 2 3 4 7 8 9 10 11 
	The active orbital indices are: 
	The virtual orbital indices are: 5 6 12 13 


CCD is also implemented as a `QCMethod`. This means that we'll have to set up an environment and a particular solver.

In [11]:
sq_hamiltonian = gqcpy.SQHamiltonian.Molecular(g_spinor_basis, molecule)

In [12]:
solver = gqcpy.CCDSolver.Plain()
environment = gqcpy.CCSDEnvironment.PerturbativeCCD(sq_hamiltonian, orbital_space)

In [13]:
def ccd0(environment):
    t2_amplitudes = environment.t2_amplitudes[-1].asTensor()
    
    dim = t2_amplitudes.shape

    new_t2_amplitudes = 0.5 * (t2_amplitudes.copy() + t2_amplitudes.copy().transpose(0,1,3,2))
    print(new_t2_amplitudes)
    environment.replace_current_t2_amplitudes(new_t2_amplitudes, dim[0], dim[2])

In [14]:
CCD0_step = gqcpy.FunctionalStep_CCSDEnvironment(ccd0, description="Function that includes the CCD0 step in the CCD algorithm.")
solver.insert(CCD0_step, 2)

In [15]:
print(solver.description())

An iterative algorithm (with a maximum of 128 iterations) consisting of the following steps:
An algorithmic step consisting of 4 algorithmic steps:
	1. Calculate the current CCD intermediates as described in Stanton1991.
	2. Calculate the new T2-amplitudes using an update formula from the current T2-amplitudes.
	3. Function that includes the CCD0 step in the CCD algorithm.
	4. Calculate the current CCD electronic correlation energy.

With the following convergence criterion:
A convergence criterion that checks if the norm of the difference of two iterates (the T2 amplitudes) is converged, with a tolerance of 1.00e-08.


Let's now proceed by actually optimizing the CCD wave function model. This may take a while if gqcp is built in Debug mode!

In [ ]:
ccd_qc_structure = gqcpy.CCD.optimize(solver, environment)

[[[[-0.000e+00  1.888e-22 -8.343e-22 -2.283e-19]
   [ 1.888e-22 -0.000e+00 -0.000e+00 -0.000e+00]
   [-8.343e-22 -0.000e+00 -0.000e+00  1.273e-34]
   [-2.283e-19 -0.000e+00  1.273e-34 -0.000e+00]]

  [[-7.660e-37 -1.273e-34 -0.000e+00  4.290e-34]
   [-1.273e-34 -0.000e+00 -0.000e+00 -0.000e+00]
   [-0.000e+00 -0.000e+00 -0.000e+00 -0.000e+00]
   [ 4.290e-34 -0.000e+00 -0.000e+00 -0.000e+00]]

  [[-0.000e+00 -0.000e+00 -0.000e+00 -0.000e+00]
   [-0.000e+00 -0.000e+00 -0.000e+00 -0.000e+00]
   [-0.000e+00 -0.000e+00 -0.000e+00 -0.000e+00]
   [-0.000e+00 -0.000e+00 -0.000e+00 -0.000e+00]]

  ...

  [[ 4.928e-05 -1.639e-02 -5.118e-16  3.321e-02]
   [-1.639e-02 -0.000e+00 -0.000e+00  2.378e-31]
   [-5.118e-16 -0.000e+00  1.680e-33  1.227e-02]
   [ 3.321e-02  2.378e-31  1.227e-02 -0.000e+00]]

  [[-0.000e+00 -0.000e+00  2.049e-02 -0.000e+00]
   [-0.000e+00  1.884e-04 -8.760e-03 -2.087e-16]
   [ 2.049e-02 -8.760e-03 -0.000e+00 -0.000e+00]
   [-0.000e+00 -2.087e-16 -0.000e+00  4.458e-18]]

  [

[[[[-5.191e-08  8.061e+08  2.202e+07 -6.079e+08]
   [ 8.061e+08 -1.110e+09  2.809e+07  1.332e+11]
   [ 2.202e+07  2.809e+07 -1.612e+09  1.881e-03]
   [-6.079e+08  1.332e+11  1.881e-03 -1.903e+12]]

  [[-5.852e+05 -2.135e-03 -5.282e-03 -6.672e+10]
   [-2.135e-03  3.530e-03  8.676e+09 -2.016e+10]
   [-5.282e-03  8.676e+09  1.162e+09  9.516e+11]
   [-6.672e+10 -2.016e+10  9.516e+11 -3.933e-07]]

  [[ 1.544e+06 -2.210e-03 -1.008e+11 -1.368e+08]
   [-2.210e-03 -5.383e-02 -8.188e+11 -6.152e+11]
   [-1.008e+11 -8.188e+11 -1.378e+08  1.783e+11]
   [-1.368e+08 -6.152e+11  1.783e+11 -3.255e+12]]

  ...

  [[ 1.755e+10  1.275e+10 -2.002e+09  1.274e+10]
   [ 1.275e+10  4.032e+10 -3.558e+09  6.482e+11]
   [-2.002e+09 -3.558e+09 -1.512e+09 -1.550e+11]
   [ 1.274e+10  6.482e+11 -1.550e+11 -4.401e+12]]

  [[-5.055e+09  4.387e+10  7.589e+10 -4.239e+03]
   [ 4.387e+10  3.891e+12  1.322e+12  2.388e+10]
   [ 7.589e+10  1.322e+12  7.027e+10  3.080e+12]
   [-4.239e+03  2.388e+10  3.080e+12  1.890e+12]]

  [

[[[[        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]]

  [[        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]]

  [[        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]]

  ...

  [[        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]]

  [[        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan    

/Users/daria/opt/anaconda3/envs/gqcp_dev/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in add
  


[[[[        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]]

  [[        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]]

  [[        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]]

  ...

  [[        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]]

  [[        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan    

...

[[[[        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]]

  [[        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]]

  [[        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]]

  ...

  [[        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]]

  [[        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan    

[[[[        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]]

  [[        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]]

  [[        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]]

  ...

  [[        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]]

  [[        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan    

[[[[       nan        nan        nan        nan]
   [       nan        nan        nan        nan]
   [       nan        nan        nan        nan]
   [       nan        nan        nan        nan]]

  [[       nan        nan        nan        nan]
   [       nan        nan        nan        nan]
   [       nan        nan        nan        nan]
   [       nan        nan        nan        nan]]

  [[       nan        nan        nan        nan]
   [       nan        nan        nan        nan]
   [       nan        nan        nan        nan]
   [       nan        nan        nan        nan]]

  ...

  [[       nan        nan        nan        nan]
   [       nan        nan        nan        nan]
   [       nan        nan        nan        nan]
   [       nan        nan        nan        nan]]

  [[       nan        nan        nan        nan]
   [       nan        nan        nan        nan]
   [       nan        nan        nan        nan]
   [       nan        nan        nan        nan]]

  [

[[[[        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]]

  [[        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]]

  [[        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]]

  ...

  [[        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]]

  [[        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan    

[[[[        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]]

  [[        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]]

  [[        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]]

  ...

  [[        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan         nan         nan]]

  [[        nan         nan         nan         nan]
   [        nan         nan         nan         nan]
   [        nan         nan    

In [ ]:
ccd_correlation_energy = ccd_qc_structure.groundStateEnergy()
print(ccd_correlation_energy)